In [48]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [49]:
#read data
train_data = pd.read_csv("train.csv")

#### 데이터 전처리

데이터 전처리 후에 editedTrain.csv 파일로 저장됨

In [50]:
# 국가를 대륙으로 매핑하는 딕셔너리
country_continent_dict = {
 'US': 'US',
 'Cuba': 'North America',
 'Portugal': 'Europe',
 'Mexico': 'North America',
 'Unknown': 'US',
 'Puerto-Rico': 'North America',
 'Germany': 'Europe',
 'Japan': 'Asia',
 'Poland': 'Europe',
 'Columbia': 'South America',
 'Philippines': 'Asia',
 'Italy': 'Europe',
 'Trinadad&Tobago': 'South America',
 'England': 'Europe',
 'South Korea': 'Asia',
 'Iran': 'Asia',
 'France': 'Europe',
 'India': 'Asia',
 'China': 'Asia',
 'Dominican-Republic': 'North America',
 'Scotland': 'Europe',
 'Ecuador': 'South America',
 'Nicaragua': 'North America',
 'Peru': 'South America',
 'Cambodia': 'Asia',
 'Canada': 'North America',
 'Jamaica': 'North America',
 'Vietnam': 'Asia',
 'Hong Kong': 'Asia',
 'Thailand': 'Asia',
 'Haiti': 'North America',
 'Guatemala': 'North America',
 'Laos': 'Asia',
 'Yugoslavia': 'Europe',
 'Ireland': 'Europe',
 'El-Salvador': 'North America',
 'Panama': 'North America',
 'Honduras': 'North America',
 'Greece': 'Europe',
 'Outlying-U S (Guam USVI etc)': 'US',
 'Hungary': 'Europe',
 'Taiwan': 'Asia',
 'Holand-Netherlands': 'Europe'
}

# 업데이트할 열 목록: 본인 출신국가, 엄마 출신국가, 아빠 출신국가
#columns_to_update = ['Birth_Country', 'Birth_Country (Mother)', 'Birth_Country (Father)']
columns_to_update = ['Birth_Country (Mother)', 'Birth_Country (Father)']


#### 내가 추가
# "Unknown"을 "US"로 대체
train_data['Birth_Country'] = train_data['Birth_Country'].replace('Unknown', 'US')
# "US"와 "Mexico"를 제외한 모든 값을 "Others"로 대체
train_data['Birth_Country'] = train_data['Birth_Country'].apply(lambda x: x if x in ['US', 'Mexico'] else 'Others')




# 각 열에 대해 국가를 대륙으로 매핑
for column in columns_to_update:
    train_data[column] = train_data[column].map(country_continent_dict)

# Remove rows where 'Age' is below 17 or above 75
train_data = train_data[train_data['Age'].between(17, 75)]

# Remove rows with 'Employment Status' as 'Not Working' or 'Seeking Full-Time'
train_data = train_data[~train_data['Employment_Status'].isin(['Not Working', 'Seeking Full-Time'])]

# Drop the 'Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status' columns
train_data.drop(['Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status'], axis=1, inplace=True)

# Map 'Gender' values from 'M' and 'F' to 0 and 1, respectively
train_data['Gender'] = train_data['Gender'].map({'M': 0, 'F': 1})


## 내가 추가 income이상치 제거 (효과 있음)
train_data = train_data[train_data['Income'] <= 2500]


## 내가 추가 군인,어린이은 그냥 제외하고 0으로 (테스트 중)
train_data = train_data[~train_data['Industry_Status'].isin(['Armed Forces', 'Not in universe or children'])]

# Consolidate education levels and rename as specified
education_map = {
    'High graduate': 'High', 'High Senior': 'High', 
    'High Junior': 'High', 'High Sophomore': 'High',
    'Elementary (5-6)': 'Elementary(1-6)', 'Elementary (1-4)': 'Elementary(1-6)',
    'Kindergarten': 'Baby', 'Children': 'Baby'
}
train_data['Education_Status'] = train_data['Education_Status'].replace(education_map)
columns=[
    'Education_Status',
    'Employment_Status',
    'Industry_Status',
    'Occupation_Status',
    'Race',
    'Hispanic_Origin',
    'Martial_Status',
    'Household_Summary',
    'Citizenship',
    'Birth_Country',
    'Birth_Country (Father)',
    'Birth_Country (Mother)',
    'Tax_Status'
]
edit_train_data=pd.get_dummies(train_data,columns=columns,dtype=int)
edit_train_data = edit_train_data.drop(columns=['ID'])
edit_train_data = edit_train_data.drop(columns=['Income'])
edit_train_data.to_csv("editedTrain.csv", index=False)

---


# 작업 영역 시작

#### RMSE를 줄이기 위해 이후 작업이 필요한 부분

#### 알맞은 모델과 파라미터를 조정해 가며 비교 필요

#### 모델 학습


In [51]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


ready_X = edit_train_data
ready_y = train_data['Income']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(ready_X, ready_y, test_size=0.2, random_state=42)

# 모델 초기화 및 학습
rf = RandomForestRegressor(n_estimators=100, random_state=50)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=50)

#### 모델 성능 평가 (Training Data 이용)

In [52]:

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV


647보다 작아야함

In [54]:
# 버전 2
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# 데이터 준비
# X, y를 적절히 정의하세요.
X=X_train
y=y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost 모델 초기화 (CPU 사용 설정)
cb_reg = CatBoostRegressor(task_type="CPU", verbose=False, random_state=42)

# RandomizedSearchCV를 사용한 하이퍼파라미터 최적화
param_dist = {
    'iterations': [100, 500, 1000],
    'learning_rate': np.logspace(-3, 0, 4),
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=cb_reg,
    param_distributions=param_dist,
    n_iter=50,  # 시도할 하이퍼파라미터 조합의 수
    cv=3,
    verbose=2,
    random_state=42,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# 최적화 수행
random_search.fit(X_train, y_train)

# 최적의 모델 찾기
best_cb_model = random_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
y_pred_cb = best_cb_model.predict(X_test)

# 성능 평가
mse_cb = mean_squared_error(y_test, y_pred_cb)
rmse_cb = np.sqrt(mse_cb)
print(f"Optimized RMSE with CatBoost: {rmse_cb}")


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.1; total time=   0.2s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.1; total time=   0.2s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.1; total time=   0.2s
[CV] END depth=4, iterations=500, l2_leaf_reg=3, learning_rate=0.001; total time=   0.6s
[CV] END depth=4, iterations=500, l2_leaf_reg=3, learning_rate=0.001; total time=   0.6s
[CV] END depth=4, iterations=500, l2_leaf_reg=3, learning_rate=0.001; total time=   0.7s
[CV] END depth=6, iterations=500, l2_leaf_reg=7, learning_rate=0.01; total time=   0.8s
[CV] END depth=6, iterations=500, l2_leaf_reg=7, learning_rate=0.01; total time=   0.8s
[CV] END depth=6, iterations=500, l2_leaf_reg=7, learning_rate=0.01; total time=   0.8s
[CV] END depth=6, iterations=1000, l2_leaf_reg=5, learning_rate=0.01; total time=   1.7s
[CV] END depth=6, iterations=1000, l2_leaf_reg=5, learnin

RSME 667 -> 실제 제출 540.24 (income 이상치 제거 안한 버전)

RSME 525 -> 실제 제출 539.42 (income 이상치 제거 3000제거 버전)

RSME 494 -> 실제 제출 539.11932 (income 이상치 제거 2500제거 버전)

---

RMSE 494.67342091188004 -> (income 이상치 제거 2500제거 버전)+(본인 출생 국가 미국과 그 외로 구분)

RMSE 494.67342091188004 -> + "Industry_Status" 열에서 "Armed Forces"인 놈은 그냥 income 0으로 

RMSE 518.6723277342746 -> + Industry_Status' 열의 값이 "Armed Forces"와 "Not in universe or children"인 행을 income 그냥 0

# 작업 영역 끝

---


#### 제출 파일 만들기

Test Data 불러오기


In [141]:
testData = pd.read_csv("EditedTest.csv")

전처리가 원료된 testData를 원핫 인코딩

In [142]:

row_to_zero = testData[(testData["Age"] < 17) | 
                      (testData["Age"] > 75) | 
                      (testData["Employment_Status"] == "Not Working") | 
                      (testData["Employment_Status"] == "Seeking Full-Time")|
                      (testData['Industry_Status'] == 'Armed Forces')|
                      (testData['Industry_Status']=='Armed Forces')|
                      (testData['Industry_Status']=='Not in universe or children')].index

# 'df_encoded'는 훈련 데이터에 대해 이미 원핫 인코딩이 수행된 DataFrame입니다.
# 'train_cols'는 원핫 인코딩된 훈련 데이터의 열 순서입니다.
train_cols = edit_train_data.columns

# 'columns' 리스트에 지정된 열에 대해 testData 데이터프레임을 원핫 인코딩합니다.
edited_testData = pd.get_dummies(testData, columns=columns, dtype=int)

# 훈련 데이터에만 있는 열을 찾습니다.
missing_cols = set(train_cols) - set(edited_testData.columns)

# 훈련 데이터에만 있는 열을 테스트 데이터에 추가하고, 해당 열의 값을 0으로 설정합니다.
for col in missing_cols:
    edited_testData[col] = 0

# 테스트 데이터에서 훈련 데이터에 없는 열을 찾습니다.
extra_cols = set(edited_testData.columns) - set(train_cols)

# 테스트 데이터에서 훈련 데이터에 없는 열을 제거합니다.
edited_testData = edited_testData.drop(columns=extra_cols)

# 테스트 데이터의 열 순서를 훈련 데이터의 열 순서와 동일하게 재정렬합니다.
edited_testData = edited_testData[train_cols]

# 이제 'edited_testData'는 원핫 인코딩된 테스트 데이터이며, 열 순서가 'df_encoded'와 일치합니다.


# 여기서 알맞은 모델을 사용해서 y_pred를 예측해야한다 (작업영역임)

예측값 생성해서 y_pred에 저장

In [143]:
y_pred = best_cb_model.predict(edited_testData) 
for i in range(len(y_pred)):
    if i in row_to_zero:
        y_pred[i]=0
        if y_pred[i]<0 :
            y_pred[i]=0

test.csv 파일에 있는 데이터를 예측하여 submissionFile.csv 파일로 저장


In [144]:
# ID 생성
ids = [f'TEST_{i:04d}' for i in range(len(y_pred))]

# 데이터프레임 생성
df = pd.DataFrame({
    'ID': ids,
    'Income': y_pred
})

# CSV 파일로 저장
df.to_csv('submissionFile.csv', index=False)